# More methods for reference ET: Hupsel 2014 data

## Intro
Until now you have solely used the Makkink method to determine the reference evapotranspiration. Although the methods originates in the Netherlands, it is gaining traction in scientific literature in the past 20 years. Methods that are internationally more commonly used are Priestley-Taylor and Penman-Monteith. In particular the latter, as it forms the basis for the standard method of the FAO to determine crop evapotranspiration (FAO: Food and Agriculture Organization of the UN). Some additional information about the various reference ET methods, and in particular the details on how to apply the Penman-Monteith equation in the context of the FAO guidelines, is given in this [overview](reference_ET_concept.pdf). Additional details on the FAO-method (not needed for this practical can be found at [FAO](http://www.fao.org/3/x0490e/x0490e00.htm).

In this exercise you will compute the reference ET with three methods (Makkink, Priestley-Taylor and Penman-Monteith), and compare those to the actual evapotranspiration. For this you will use the 2014 Hupsel data (daily means) again.

This practical does not have a separate **answer sheet**. You should use the answer sheet of Practical 3a.

## The method
The main steps of the analysis of this exercise are:
1. Compute the reference ET with three methods, based on daily mean data
2. Compare the results of those three methods between each other
3. Compare the results of the three methods with the actual evapotranspiration
In the last two steps you will certainly find differences. Those difference may well be related to the specific weather conditions or surface conditions. Hence, your analysis of the weather conditions during May 2014, which you made in Practical-1, will come in handy.

Some of the steps you have done in Practical-1 already, namely computing ET<sub>ref</sub> with Makkink and computing ET<sub>act</sub> from the eddycovariance data. There is no need to copy over the results from Practical-1 as you will easily reproduce them here. 

Hence the main tasks will be to 
* implement functions for the Priestley-Taylor and Penman-Monteith method
* do the comparison



## The data
The data that you will use here are identical to the data you used in Practical-1. As before, in these final steps we will focus on daily mean data (i.e. data that have been averaged over 24 hours).

## Initialize Python stuff and read the data
Please run the cell below by selecting it and pressing Shift+Enter. Or Press the Run button in the toolbar at the top of the screen (with the right pointing triangle).

In [ ]:
# Load some necessary Python modules
import pandas as pd # Pandas is a library for data analysis
pd.set_option("mode.chained_assignment", None)
import numpy as np # Numpy is a library for processing multi-dimensional datasets
from hupsel_helper import myplot, myreadfile
from hupsel_helper import f_Lv, f_esat, f_s, f_gamma, f_cp, f_makkink, check_ET, check_PT, check_ra, check_PM

With the commands above, the following functions have become available (only `f_cp` is new):
* `f_Lv(T)`: compte latent heat of vapourization from temperature (in K)
* `f_esat(T)`: compute saturated vapour pressure from temperature (in K)
* `f_s(T)`: compute the slope of the saturated vapour pressure as a function of temperature (in K)
* `f_gamma(T, p, q)`: compute the psychrometer constant from temperature (K), pressure (Pa) and specific humidity (kg/kg)
* `f_cp(q)`: compute the specific heat of air (in J/kg/K) using specific humidity (in kg/kg)
* `f_makkink(Kin, T, p, q)`: compute reference evapotranspiration according to the Makkink equation.

We have added some new possibilities to the `myplot` function (already introduced in practical 3a):
* you can now manually set the label text on the x-axis and y-axis (in particular relevant if you do not plot from a dataframe): `myplot( [x, y], xlabel='wind speed (m/s)', ylabel='temperature (K)')`
* you can now manually set the name of a series, to be used in the legend (again, relevant if no dataframe is used): `myplot( [x, y, 'o', 'my specially constructed variable'] )`. Note that in this case you *should* specify the type of plotting symbol (here a dot: `'o'`, could also be `'-'` for line and `'#'` for a bar graph).

Now read the data for the current year from the Excel file.

In [ ]:
# File name: this is a different file that you worked on before
fname='Hupsel2014_MeteoData.xlsx'

# Get the data
df = myreadfile(fname)

## Prepare the functions to compute ET<sub>ref</sub>
       

### <span style='background:lightblue'>Question 1a</span>
Construct your function to compute ET<sub>ref</sub> according to **Priestley-Taylor**. This method is quite close to the Makkink method, so the function you make here will look similar to the one you made for the Makkink method in practical 1. Once you wrote your function, do not forget to press Shift+Enter to run the code. 

In [ ]:
# Function to compute reference evapotranspiration according to Priestley-Taylor
# Input
#    Q_net : net radiation (W/m2)
#    G     : soil heat flux (W/m2)
#    T     : temperature (Kelvin)
#    p     : pressure (Pa)
#    q     : specific humidity (kg/kg)
# Output
#    LvEref: reference evapotranspiration according to Priestley-Taylor (W/m2)
#
# See secton 7.7 of the AVSI formularium, chapter 7 in Moene & van Dam (2014), 
# or the supporting document linked to at the intro of this exercise.
# Please note what is the unit of the resulting number !
def f_PT(Q_net, G, T, p, q):
    # First compute s and gamma from the data (those functions have already been defined)
    gamma = f_gamma(T, p, q)
    s = f_s(T)
    
    # Now construct the Priestley-Taylor equation (i.e. replace the '0' by the correct equation)
    # What is the unit?
    result  = 0
    
    return result

Now test the function in the cell below. Both test the structure of your function with the function `check_PT(f_PT)`, and check the resulting values (do they make physical sense?).

In [ ]:
# Check the structure of the function (uncomment the next line: remove to '#' in front of it to run it)
# check_PT(f_PT)
#
# Check whether the resulting values make sense: pass the data of the current dataset to the function
# (e.g. df['Q_net_m']; make sure that the units of the variables are the units that the function expects!)
# Complete the line of code below and uncomment it.
# test_data = f_PT(df['Q_net_m'],... )
# print(test_data)

### <span style='background:lightblue'>Question 1b</span>

Next, construct the function to compute ET<sub>ref</sub> according to **Penman-Monteith**. This function is somewhat more complex than what you did before. In particular, you will need a function that determines the aerodynamic resistance from wind speed.

In [ ]:
# Function to compute aerodynamic resistance for neutral conditions (consistent with the 
# FAO method)
# Input
#    u     : wind speed (m/s)
#    zu    : height at which wind speed was measured (m)
#    zT    : height at which temperature and humidity was measured (m)
#    d     : displacement height (m)
#    z0    : roughness length for momentum
#    z0h   : roughness length for heat
# Output
#    ra    : aerodynamic resistance (s/m)
#
# See he supporting document linked to at the intro of this exercise.
# See secton 7.3 of the AVSI formularium, chapter 7 in Moene & van Dam (2014), 
def f_ra(u, zu, zT, d, z0, z0h):
    # Note: you will need the natural logarithm (which we write in math as ln(x)).
    # In Python this is a function from the numpy library (imported as np: it is called log. Hence
    # the natural log of x would be computed as: answer = np.log(x)
    
    # Compute the aerodynamic resistance (i.e. replace the zero by an appropriate expression)    
    result = 1
    
    return result

In [ ]:
# Check the structure of the function (uncomment the next line: remove to '#' in front of it to run it)
# check_ra(f_ra)
#
# Check whether the resulting values make sense: pass the data of the current dataset to the function
# (e.g. df['u']; make sure that the units of the variables are the units that the function expects!)
# Complete the line of code below and uncomment it. The different heights you can enter as a single number.
# test_data = f_ra(df['u_10'], ... )
# print(test_data)

In [ ]:
# Function to compute reference evapotranspiration according to Penman-Monteith
# Input
#    Q_net : net radiation (W/m2)
#    G     : soil heat flux (W/m2)
#    T     : temperature (Kelvin)
#    p     : pressure (Pa)
#    q     : specific humidity (kg/kg)
#    ra    : aerodynamic resistance (s/m)
#    rc    : canopy resistance (s/m)
# Output
#    LvEref: reference evapotranspiration according to Penman-Monteith (W/m2)
#
# See secton 7.7 of the AVSI formularium, chapter 7 in Moene & van Dam (2014), 
# or the supporting document linked to at the intro of this exercise.
# Please note what is the unit of the resulting number !
def f_PM(Q_net, G, T, p, q, ra, rc):
    # First compute s, gamma and cp from the data 
    # (those functions have already been defined, f_cp is new, but we made it for you)
    gamma = f_gamma(T, p, q)
    s = f_s(T)
    cp = f_cp(q)
    
    # In Penman-Monteith we use vapour pressure (e_a) as the variable for water vapour
    # content. We already have specific humidity (q) available within the function, 
    # so e_a can be determined to a reasonable approximation from q = (Rd/Rv) * (e_a/p)
    # (check the formularium how to do this)
    e_a = 0
    
    # In Penman-Monteith we need the air density (rho_a). Using the gas law we can determine
    # rho_a using pressure, temperature: p = rho_a * R * T (where R is the gas constant for humid
    # air)
    # again: see the formularium)
    rho_a = 0
       
    # Now construct the Penman-Monteith equation (i.e. replace the '0' by the correct equation)
    # It can be helpful to split the equation in a number of chunks (e.g. compute the denominator 
    # first) combine those chunks at the end
    # What is the unit?
    result  = 0
    
    return result

In [ ]:
# Check the structure of the function (uncomment the next line: remove to '#' in front of it to run it)
# check_PM(f_PM)
#
# Check whether the resulting values make sense: pass the data of the current dataset to the function
# (e.g. df['Q_net_m']).
# For the test you also need to provide values for the resistances ra and rc. 
# * For rc, use the value(s) that is/are prescribed in the FAO method (see the overview document linked to in 
#   the introduction) 
# * For ra, use your function and run it with data from the current dataset and pass that array to the PM function
#
# Complete the lines of code below and uncomment it.
# rc = ..
# ra = f_ra(df['u'], ....)
# test_data = f_PM(df['Q_net_m'], ..., ra, rc)
# print(test_data)

## Determine the reference evapotranspiration with the different methods


### <span style='background:lightblue'>Question 2</span>
Determine the reference evapotranspiration for the 2014 dataset in mm/day using the three methods that are available:
* Makkink (use the function that we provide)
* Priestley-Taylor (using the function you just made)
* Penman-Monteith according to the FAO method (using the function you just made). Note that for PM you need to provide the two resistances (canopy resistance and aerodynamic resistance) as well.

In [ ]:
# Use this cell to compute the three time series (including the unit conversion)
# Check your results with the check_ET( ) function; and also use your own insights to see if the values are reasonable.


### <span style='background:lightblue'>Question 3</span>
Explore the variation of the ET values of all methods in time (e.g. plot all three methods in one graph). Focus on:
* how do each of the methods vary, and can you link those variations to the  meteorological conditions (discussed in the first practical, last week).
* how do the three methods differ (in which direction, at which moments) and which properties of the different methods could explain those difference?

## Compare the different ET<sub>ref</sub>  methods to the actual ET
Reference ET methods are often 'sold' as giving a good estimate of the actual evapotranspiration of well-watered short vegetation. For the Makkink method we've already seen that this is not always the case for the dataset under investigation. Now it is time to see how the other methods are doing.

### <span style='background:lightblue'>Question 4</span>
Determine (again) the actual evapotranspiration in mm/day for the Hupsel data set from the eddy-covariance data (variable `LvE_m`). Now compare the ET<sub>ref</sub> given by the three methods to the actual ET of the grass. 
* Are the results of Priestley-Taylor or Penman-Monteith closer to the actual ET? 
* On which moments?
* What could be the explanation?

Again, take into account both the structure of the various methods and the meteorological conditions.

### <span style='background:lightblue'>Question 5 (sort of)</span>
Now upload your answer document(Word file) to Brightspace.

## Conclusion
With this, you've come to the end of the exercises for this Monday. You now have a solid overview of how various reference ET methods work and how they compare to the actual evapotranspiration of well-watered short grass.


## Up to the next exercise
In the next exercise (on Wednesday) we will shift from daily mean data to half-hourly data. With those data we will focus on a number of the parameters occurring in Penman-Monteith that for you took for granted: albedo, roughness length and canopy resistance.